# Community Day 2024 Supply Chain Security Tutorial
This years tutorial is about using Chainloop for attestions and combining that with SignServer to sign the attestations. EJBCA is used to issue certificates for the signing and TLS. 

This tutorial will walk you through the following:
- Building a container for this tutorial
- Generating a CycloneDX Software Bill of Materials (SBOM) using Trivy
- Scanning the container for CVE's using Trivy
- Pushing the container to an Open Container Initiative (OCI) ZOT registry using Skopeo
- Sign the container using Cosgin and Signserver
- Add an OCI registry to Chainloop
- Create a workflow and add policy to the workflow in Chainloop
- Create an attestation in Chainloop
- Sign the Chainloop attestation with Chainloop

The virtual machine that is used for this tutorial has all the pieces/components in place to complete this tutorial. This VM is meant for testing and should not be used in a production environment.

## Build the simple-todo container
The first part of this journey starts with building the container with Docker. Follow the steps below to build and test the simple-todo container.
1. Build the container with Docker:

In [ ]:
docker build -t registry.community2024.local/simple-todo -f simple-todo/Dockerfile simple-todo

2. Start the container for simple validation test:

In [ ]:
docker run -d --rm  --name todos -p 12345:12345 registry.community2024.local/simple-todo

3. Use the curl command to test the container:

In [ ]:
curl -s localhost:12345/todos | jq .

4. Stop the container:

In [ ]:
docker container stop todos

## Create SBOM and check CVE's with Trivy
After the container is built the SBOM can be created and the container can also be scanned for CVE's before pushing the container to a registry. Follow the steps below to generate a SBOM and scan for CVE's using Trivy

1. Create SBOM for the container built:

In [ ]:
trivy image --format cyclonedx --output sbom-todos.json registry.community2024.local/simple-todo

2. Scan the container to generate the CVE report:

In [ ]:
trivy image -f sarif --output todos.sarif.json registry.community2024.local/simple-todo

> **_NOTE:_** If you want to review the CVE's in standard out: `trivy image registry.community2024.local/simple-todo`

## Start the Chainloop & Keyfactor Containers
The simple-todo container is now ready to push to the registry. Before the cotnainer can be pushed the supporting infrastructure must be started. Use the command below to start the infrastrucutre for this tutorial.

1. Start the containers for the tutorial:


In [ ]:
docker compose -f compose.labs.yaml up -d

## Push the container to the OCI ZOT registry
The container was built using Docker, but Docker does not completely follow the OCI specification. Therefore we need to use another tool that can push the container built with Docker to the OCI registry. Docker is used to export the container to file, and the Skopeo tool is used to push the container to the registry. Follow the steps below to add the container to the ZOT registry.

1. Export the container from Docker to a tar file:

In [ ]:
docker image save -o todos.tar registry.community2024.local/simple-todo

2. Login to the ZOT registry with Skopeo:

In [ ]:
skopeo login -u user -p foo123 registry.community2024.local

3. Push the container to the ZOT registry:

In [ ]:
skopeo copy --format=oci --dest-tls-verify=true docker-archive:todos.tar  docker://registry.community2024.local/simple-todo

4. Open another tab and navigate the ZOT registry **`https://registry.community2024.local`** where you can see the container available in the registry.

## Sign the container with Cosign & SignServer
You may have noticed from viewing the container in the registry there is an icon to inform if the container is signed or not signed. We will now sign the container following the steps below using Cosign and SignServer.

1. Generate the container metadata payload describing the container image:

In [ ]:
cosign generate registry.community2024.local/simple-todo:latest > simple-todo-payload.json

2. Review the contents of the `simple-todo-payload.json` file:

In [ ]:
jq . simple-todo-payload.json

3. Sign the `simple-todo-payload.json` with SignSever using the curl command:

In [ ]:
curl -sF workerName=PlainSigner -F file=@simple-todo-payload.json \
--output simple-todo-payload.json.sig \
https://sign.community2024.local/signserver/process

4. Convert the signature from binary to base64:

In [ ]:
cat simple-todo-payload.json.sig | base64 > simple-todo-payload.json.sig.b64

5. Attach the signed metadata payload to the simple-todo image in the ZOT-registry using Cosign:

In [ ]:
cosign attach signature --payload simple-todo-payload.json \
--signature simple-todo-payload.json.sig.b64 \
registry.community2024.local/simple-todo:latest

6. Clean up the signature files now the signature was added to the container image:

In [ ]:
rm -f simple-todo-payload.json*

## Configure Chainloop
Now the time comes to attest to what has been built and what to include in the build attestation. Before we can begin using Chainloop we need to obtain the authentication token and add the ZOT-registry where Chainloop will store all attestation artificts.

1. Obtain the authentication token and populate the token in the chainloop client config file:

In [ ]:
TOKEN=$(docker logs chainloop-lab-control-plane-1 2>&1 | grep "headless session:" | jq -r .msg | cut -d ':' -f 2 | xargs)
sed -i "s/TKHOLDER/$TOKEN/g" /home/user/.config/chainloop/config.toml

2. Create a default organization in Chainloop:

In [ ]:
chainloop org create --name community-day-2024

3. Add a the ZOT-registry to the Chainloop Content Addressable Storage (CAS) backend:

In [ ]:
chainloop cas-backend add oci \
    --name zot-registry \
    --repo registry.community2024.local/chainloop-devel \
    --username "user" \
    --password "foo123" \
    --default

## Create a Chainloop workflow
The backend storage and token are in place to use Chainloop for this tutorial. Next we need to create a workflow in Chainloop. Think of a workflow as part the pipeline for the automated build of your project, container, etc. The workflow can be associated with a contract that specifies rules for building. Such rules for example could be requiring a container to be part of the build or the SBOM. This is a very strong feature that should be explored further outside of this tutorial. Follow the steps below to create a workflow and associate a contract with the workflow.

1. Create the workflow for the simple-todo container:

In [ ]:
chainloop workflow create --name build-release --project simple-todo

2. Update the contract for the newly created build-release workflow:

In [ ]:
chainloop wf contract update --name simple-todo-build-release --contract chainloop-contract.yaml

3. Create the API token that would be given to the development team to integrate the build system with Chainloop:

In [ ]:
export CHAINLOOP_TOKEN=$(chainloop org api-token create --name simple-todo-api-token -o token)

> **_NOTE:_** To revoke the token when done and use the command: `chainloop org api-token revoke --name simple-todo-api-token`

## Create the attestation for the simple-todo container
A workflow with a contract is now in place for the simple-todo project. Now we can create an attestation and add the material used to the attestation. To complete the attestation we will use SignServer to sign and Chainloop will publish this information to the ZOT-registry. Follow the steps below to begin the attestation.

1. Initialize the attestation for the simple-todo workflow:

In [ ]:
chainloop att init --name build-release

> **_NOTE:_** To reset the attestion to start over or cancel one in progress: `chainloop att init --name build-release --replace`

2. Add the simple-todo container to the attestation:

In [ ]:
chainloop att add --name container --value registry.community2024.local/simple-todo

3. Add the simple-todo container SBOM to the attestation:

In [ ]:
chainloop att add --value sbom-todos.json

4. Add the simple-todo container SARIF report to the attestation:

In [ ]:
chainloop att add --value todos.sarif.json

5. Before we complete the attestation lets review what is in the attestation:

In [ ]:
chainloop att status --full

6. Complete the attestation by signing the attestation with SignServer and pushing the artifacts to the ZOT-registry:

In [ ]:
chainloop attestation push --key signserver://sign.community2024.local/PlainSigner --signserver-ca-path container-configs/nginx/tls/ca.crt

7. Copy the attestion ID from the output above and paste after the `ATT_ID=` in the box below to verify the signature on the signed attestation:

In [ ]:
EXPORT ATT_ID=
chainloop workflow run describe --id "$ATT_ID" --verify true --cert PlainSigner.pem --cert-chain container-configs/nginx/tls/ca.crt

## Shutdown the containers
Now that you have completed the labs you can shut down the containers. Use the step below to shutdown all the runnint containers.

1. Shutdown the containers with Docker compose:

In [ ]:
docker compose -f compose.labs.yaml down

This completes the Community Day 2024 Supply Chain Security tutorial with Chainloop and Keyfactor. If you have any questions you can reach us at:
- Sven Rajala sven.rajala@keyfactor.com
- Jose jiparis@chainloop.dev
- Miguel miguel@chainloop.dev
- Daniel daniel@chainloop.dev